# ETL

Importo las librerias que voy a utilizar

In [1]:
import pandas as pd
import ast
import json
from nltk.corpus import stopwords
from textblob import TextBlob
import re

Ingesto el archivo 'outputs_steam_games.json'

In [2]:
df_games = pd.read_json('datasets/output_steam_games.json', lines=True)

Visualizo el DF

In [3]:
df_games.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
0,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,76561197970982479,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277.0


In [4]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   reviews_url     32133 non-null  object 
 8   discount_price  225 non-null    float64
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  float64
 12  id              32133 non-null  float64
 13  metascore       2677 non-null   object 
 14  developer       28836 non-null  object 
 15  user_id         88310 non-null  object 
 16  steam_id        88310 non-null  float64
 17  items           88310 non-nul

Elimino las filas que no contienen 'id'

In [5]:
df_games.dropna(subset=['id'], inplace=True)

Verifico si existen id duplicados

In [6]:
df_games.id.duplicated().sum()

1

Elimino los duplicados

In [7]:
df_games.drop_duplicates(subset="id", inplace=True)

Visualizo df_games

In [8]:
df_games.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,0.0,761140.0,None,Kotoshiro,None,NaN,None,NaN


Las columnas 'app_name' y 'title' contienen informacion similar, relleno los nulos con los datos de las otras columnas

In [11]:
(df_games['app_name'] == df_games['title']).sum()

31576

Relleno los faltantes con los datos de la otra columna

In [10]:
df_games['app_name'].fillna(df_games['title'], inplace= True)
df_games['title'].fillna(df_games['app_name'], inplace= True)

Elimino las columnas que considero irrelevantes o por contener muchos nulos

In [12]:
df_games.drop(columns=['publisher',
                       'app_name',
                       'url',
                       'tags',
                       'reviews_url',
                       'discount_price',
                       'specs',
                       'early_access',
                       'metascore',
                       'developer',
                       'user_id',
                       'steam_id',
                       'items',
                       'items_count'], inplace=True)

Cambio el tipo de fecha de la columna'release_date'

In [13]:
df_games['release_date'] = df_games['release_date'].apply(lambda x: pd.to_datetime(x , errors='coerce'))

C:\Users\dines\AppData\Local\Temp\ipykernel_15328\3855999810.py:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_games['release_date'] = df_games['release_date'].apply(lambda x: pd.to_datetime(x , errors='coerce'))


Elimino las filas donde existen datos nulos en la columnas 'genres' y 'release_date'

In [14]:
df_games.dropna(subset=['genres', 'release_date'], inplace=True)

In [15]:
df_games['price'].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 3.99, 9.99, 18.99,
       29.99, None, 10.99, 2.99, 1.5899999999999999, 14.99, 1.99, 59.99,
       8.99, 6.99, 7.99, 39.99, 'Free', 19.99, 7.49, 12.99, 5.99, 2.49,
       15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99,
       79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 109.99, 149.99, 771.71,
       'Install Now', 21.99, 89.99, 'Play WARMACHINE: Tactics Demo', 0.98,
       139.92, 4.29, 64.99, 'Free Mod', 54.99, 74.99, 'Install Theme',
       0.89, 'Third-party', 0.5, 'Play Now', 299.99, 1.29, 119.99, 3.0,
       15.0, 5.49, 23.99, 49.0, 20.99, 10.93, 1.3900000000000001,
       'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0, 4.0,
       1.9500000000000002, 1.5, 199.0, 189.0, 6.66, 27.99, 129.99, 179.0,
       26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 22.99, 320.0, 38.85,
       71.7, 995.0, 27.49, 3.39, 6.0, 19.95, 499.99, 199.99, 16.06, 4.68,
 

Hay valores en la columna 'price' que contienen 'str', en su gran mayoria indican que son 'Free', los reemplazo por 0.0, luego cambio el tipo de dato a float

In [16]:
df_games['price'] = df_games['price'].apply(lambda x: 0.0 if isinstance(x,str) else x)
df_games['price'] = df_games['price'].astype('float')

Cambio el nombre de la columna id para normalizar y poder relacionar el DF con otros.

In [17]:
df_games.rename(columns={'id':'item_id'}, inplace=True)

La columna 'item_id' es de tipo float, la cambio a int para quitar el .0 y luego a str ya que no es un dato de agregacion

In [18]:
df_games['item_id'] = df_games['item_id'].fillna(0).astype(int)
df_games['item_id'] = df_games['item_id'].astype(str)

Reseteo ei indice de las filas

In [19]:
df_games.reset_index(drop=True, inplace=True)

Visualizo el DF final

In [20]:
df_games.head()

,genres,title,release_date,price,item_id
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,0.00,643980
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,0.00,670290
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400
4,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,3.99,772540


In [21]:
savefile_path = 'datasets/games_1.parquet'
df_games.to_parquet(savefile_path, engine='pyarrow')

# Desanido la columna 'Genres'

En primer lugar cambio el tipo de dato a str

In [22]:
df_games['genres']  = df_games['genres'].astype(str)

Elimino corchetes, comillas y espacion vacios

In [23]:
df_games['genres']  = df_games['genres'].str.replace('[', '').str.replace(']', '')
df_games['genres']  = df_games['genres'].str.replace("'", '').str.replace("'", '')
df_games['genres']  = df_games['genres'].str.replace(' ', '').str.replace(' ', '')
df_games['genres'].str.strip()

0          Action,Casual,Indie,Simulation,Strategy
1                    FreetoPlay,Indie,RPG,Strategy
2        Casual,FreetoPlay,Indie,Simulation,Sports
3                          Action,Adventure,Casual
4                      Action,Adventure,Simulation
                           ...                    
28653                Action,Adventure,Casual,Indie
28654             Casual,Indie,Simulation,Strategy
28655                        Casual,Indie,Strategy
28656                      Indie,Racing,Simulation
28657                                 Casual,Indie
Name: genres, Length: 28658, dtype: object

Genéro un nuevo DF dummies

In [24]:
dummies = df_games['genres'].str.get_dummies(sep=',')

Concateno 'df_games' y 'dummies'

In [25]:
df_games = pd.concat([df_games, dummies], axis=1)

In [26]:
dummies.head(20)

,Accounting,Action,Adventure,Animation&amp;Modeling,AudioProduction,Casual,Design&amp;Illustration,EarlyAccess,Education,FreetoPlay,...,PhotoEditing,RPG,Racing,Simulation,SoftwareTraining,Sports,Strategy,Utilities,VideoProduction,WebPublishing
0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0
2,0,0,0,0,0,1,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
3,0,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
7,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
8,0,0,0,0,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
9,0,1,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


Elimino la columna 'genres'

In [27]:
df_games.drop(['genres'], axis=1, inplace=True)

Guardo df_games como games.parquet

In [ ]:
savefile_path = 'datasets/games.parquet'
df_games.to_parquet(savefile_path, engine='pyarrow')

===========================================================================================================================

Ingesto el archivo australian_users_items.json

In [28]:
rows = []
with open('datasets/australian_users_items.json', encoding='MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))
        
df_items = pd.DataFrame(rows)

In [29]:
df_items.head(5)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [30]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


Verifico si existen users_id duplicados

In [31]:
df_items['user_id'].duplicated().sum()

684

Visualizo algunos duplicados para ver si contienen informacion identica

In [32]:
df_items[df_items['user_id'] == 'nitrorupe']

,user_id,items_count,steam_id,user_url,items
3464,nitrorupe,147,76561198075283333,http://steamcommunity.com/id/nitrorupe,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
3466,nitrorupe,147,76561198075283333,http://steamcommunity.com/id/nitrorupe,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


In [33]:
df_items[df_items['user_id'] == 'yeahrightiom8']

,user_id,items_count,steam_id,user_url,items
3973,yeahrightiom8,0,76561198081031350,http://steamcommunity.com/id/yeahrightiom8,[]
3974,yeahrightiom8,0,76561198081031350,http://steamcommunity.com/id/yeahrightiom8,[]


In [34]:
df_items[df_items['user_id'] == 'Skydogsgaming']

,user_id,items_count,steam_id,user_url,items
4840,Skydogsgaming,17,76561198261018670,http://steamcommunity.com/id/Skydogsgaming,"[{'item_id': '8930', 'item_name': 'Sid Meier's..."
4841,Skydogsgaming,17,76561198261018670,http://steamcommunity.com/id/Skydogsgaming,"[{'item_id': '8930', 'item_name': 'Sid Meier's..."


Elimino los 'user_id' duplicados

In [35]:
df_items.drop_duplicates(subset="user_id", inplace=True)

Cambio el tipo de dato de la columna 'user_id'

In [36]:
df_items['user_id'] = df_items['user_id'].astype(str) 

Evaluo la columna items, cada campo contiene una lista de diccionarios

In [37]:
df_items['items'][0]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 7,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '60',
  'item_name': 'Ricochet',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '70',
  'item_name': 'Half-Life',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '130',
  'item_name': 'Half-Life: Blue Shift',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '300',
  'item_name': 'Day of Defeat: Source',
  'playtime_forever': 4733,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: S

Cuento las listas vacias en la columna 'items'

In [38]:
df_items['items'].apply(lambda x: x == []).sum()

16714

Elimino las filas con listas vacias en la columna 'items'

In [39]:
df_items = df_items[df_items['items'].apply(lambda x: x != [])]

La columna items contiene una lista de diccionarios hay que desanidar, primero separamos los datos de cada fila del DataFrame y luego voy creando las nuevas filas en diccionarios los cuales agrego a una lista que luego voy a convertir en el nuevo datafame aplanando la estructura de la columna items

In [40]:
items_des = []

for index, row in df_items.iterrows():
    user_id = row['user_id']
    items_count = row['items_count']
    steam_id = row['steam_id']
    user_url = row['user_url']
    items = row['items']
    
    for i in items:   
        new_row = {
        'user_id': user_id,
        'items_count': items_count,
        'steam_id' : steam_id,
        'user_url' : user_url,
        'item_id': i.get('item_id', ''),
        'item_name': i.get('item_name', ''),
        'playtime_forever': i.get('playtime_forever', ''),
        'playtime_2weeks': i.get('playtime_2weeks', '')
        }
        
        items_des.append(new_row)

df_items_des = pd.DataFrame(items_des)

In [41]:
df_items_des.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0,0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0,0


Elimino columnas que considero irrelevantes

In [42]:
df_items_des.drop(columns=['items_count',
                           'steam_id',
                           'user_url',
                           'item_name',
                           'playtime_2weeks'], inplace=True)

Cuento cuantos 0 hay en la columna 'playtime_forever'

In [43]:
df_items_des['playtime_forever'].apply(lambda x: x == 0).sum()

1847730

In [46]:
a = df_items_des['item_id'].unique()
len(a)

10050

Elimino las filas que tienen 0 en la columna 'playtime_forever'

In [45]:
df_items_des =  df_items_des[df_items_des['playtime_forever'] != 0]

Visualizo el tamaño del DF final

In [47]:
df_items_des.shape

(3246352, 3)

Reseteo el indice de filas

In [48]:
df_items_des.reset_index(drop=True, inplace=True)

Guardo el DF como items.parquet

In [ ]:
savefile_path = 'datasets/items.parquet'
df_items_des.to_parquet(savefile_path, engine='pyarrow')

===========================================================================================================================

Ingesto el archivo 'australian_user_reviews.json'

In [49]:
rows = []
with open('datasets/australian_user_reviews.json', encoding='utf-8') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))
        
df_reviews = pd.DataFrame(rows)

In [50]:
dis = df_reviews['user_id'].unique()
len(dis)

25485

In [51]:
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [52]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


Busco 'user_id'duplicados

In [53]:
df_reviews.user_id.duplicated().sum()

314

Visualizo los duplicados

In [54]:
dupl_reviews = df_reviews[df_reviews['user_id'].duplicated()]
dupl_reviews.head(20)

,user_id,user_url,reviews
456,bokkkbokkk,http://steamcommunity.com/id/bokkkbokkk,"[{'funny': '', 'posted': 'Posted September 24,..."
1182,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,"[{'funny': '', 'posted': 'Posted January 10, 2..."
1456,76561198062039159,http://steamcommunity.com/profiles/76561198062...,"[{'funny': '', 'posted': 'Posted August 24, 20..."
1477,76561198045009232,http://steamcommunity.com/profiles/76561198045...,"[{'funny': '', 'posted': 'Posted October 31, 2..."
1746,nitr0ticwolf,http://steamcommunity.com/id/nitr0ticwolf,"[{'funny': '', 'posted': 'Posted December 12, ..."
1747,nitrorupe,http://steamcommunity.com/id/nitrorupe,"[{'funny': '', 'posted': 'Posted October 10, 2..."
1853,76561198083347489,http://steamcommunity.com/profiles/76561198083...,"[{'funny': '', 'posted': 'Posted January 15.',..."
1883,Ppkdoevil,http://steamcommunity.com/id/Ppkdoevil,"[{'funny': '', 'posted': 'Posted June 25.', 'l..."
2204,camo88,http://steamcommunity.com/id/camo88,"[{'funny': '', 'posted': 'Posted January 21, 2..."
2283,64512328,http://steamcommunity.com/id/64512328,"[{'funny': '', 'posted': 'Posted January 26, 2..."


Verifico si los duplicados poseen informacion identica

In [55]:
df_reviews[df_reviews['user_id'] == 'nitrorupe']

,user_id,user_url,reviews
1745,nitrorupe,http://steamcommunity.com/id/nitrorupe,"[{'funny': '', 'posted': 'Posted October 10, 2..."
1747,nitrorupe,http://steamcommunity.com/id/nitrorupe,"[{'funny': '', 'posted': 'Posted October 10, 2..."


In [56]:
compara = df_reviews['reviews'].iloc[1745] == df_reviews['reviews'].iloc[1747] 
compara

True

In [57]:
df_reviews[df_reviews['user_id'] == 'Skydogsgaming']

,user_id,user_url,reviews
2446,Skydogsgaming,http://steamcommunity.com/id/Skydogsgaming,"[{'funny': '', 'posted': 'Posted August 29.', ..."
2447,Skydogsgaming,http://steamcommunity.com/id/Skydogsgaming,"[{'funny': '', 'posted': 'Posted August 29.', ..."


In [58]:
compara1 = df_reviews['reviews'].iloc[2446] == df_reviews['reviews'].iloc[2447] 
compara1

True

In [59]:
df_reviews[df_reviews['user_id'] == '76561198094665607']

,user_id,user_url,reviews
2665,76561198094665607,http://steamcommunity.com/profiles/76561198094...,"[{'funny': '1 person found this review funny',..."
2667,76561198094665607,http://steamcommunity.com/profiles/76561198094...,"[{'funny': '1 person found this review funny',..."


In [60]:
compara2 = df_reviews['reviews'].iloc[2665] == df_reviews['reviews'].iloc[2667] 
compara2

True

Elimino los duplicados, ya que considero que todos contienen información identica

In [61]:
df_reviews.drop_duplicates(subset="user_id", inplace=True)

Cambio el tipo de dato de la columna 'user_id'

In [62]:
df_reviews['user_id'] = df_reviews['user_id'].astype(str)

Visualizo el contenido de los campos de las columna 'reviews'

In [63]:
df_reviews['reviews'][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

Desanido la columna 'reviews'

In [64]:
reviews_desanidado = []

for index, row in df_reviews.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    reviews = row['reviews']
    
    for i in reviews:   
        newrow = {
        'user_id': user_id,
        'user_url' : user_url,
        'funny': i.get('funny', ''),
        'posted': i.get('posted', ''),
        'last_edited': i.get('last_edited', ''),
        'item_id': i.get('item_id', ''),
        'helpful': i.get('helpful', ''),
        'recommend': i.get('recommend', ''),
        'review': i.get('review', '')
        }
        
        reviews_desanidado.append(newrow)

df_reviews_des = pd.DataFrame(reviews_desanidado)

In [65]:
df_reviews_des.head(10)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [66]:
df_reviews_des.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58430 entries, 0 to 58429
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      58430 non-null  object
 1   user_url     58430 non-null  object
 2   funny        58430 non-null  object
 3   posted       58430 non-null  object
 4   last_edited  58430 non-null  object
 5   item_id      58430 non-null  object
 6   helpful      58430 non-null  object
 7   recommend    58430 non-null  bool  
 8   review       58430 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.6+ MB


# Análisis de sentimientos - Preparación

Elimino preposiciones de cada str en la columna 'review' en los diferentes idiomas en las que se realizaron las reviews

In [67]:
stop_words = set(stopwords.words('english'))
df_reviews_des['review'] = df_reviews_des['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

In [68]:
stop_words = set(stopwords.words('spanish'))
df_reviews_des['review'] = df_reviews_des['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

In [69]:
stop_words = set(stopwords.words('portuguese'))
df_reviews_des['review'] = df_reviews_des['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

Las reviews tambien contienen emojis y caracteres especiales con el modulo re los elimino

In [70]:
df_reviews_des['review'] = df_reviews_des['review'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

Se crea la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo

In [71]:
# Limpio los datos
df_reviews_des['review'] = df_reviews_des['review'].str.lower()
df_reviews_des['review'] = df_reviews_des['review'].str.replace('[^a-zA-Z0-9]', ' ')
# Aplico análisis de sentimiento
df_reviews_des['sentiment'] = df_reviews_des['review'].apply(lambda x: TextBlob(x).sentiment.polarity)
# Creo la columna 'sentiment_analysis'
df_reviews_des['sentiment_analysis'] = df_reviews_des['sentiment'].apply(lambda x: 0 if x < 0 else 1 if x == 0 else 2)

Reemplazo los valores booleanos en la columna 'recommend por 1 y 0 para una mejor performance en el analisis EDA y modelo de ML

In [72]:
df_reviews_des['recommend'] = df_reviews_des['recommend'].map({True : 1, False : 0})

Cambio el tipo de dato de la columna 'posted por uno del tipo datetime

In [73]:
df_reviews_des['posted'] = df_reviews_des['posted'].str.replace('Posted', '')
df_reviews_des['posted'] = df_reviews_des['posted'].str.replace(',', '')
df_reviews_des['posted'] = df_reviews_des['posted'].str.replace('.', '')
df_reviews_des['posted'] = pd.to_datetime(df_reviews_des['posted'], errors= 'coerce')

In [76]:
df_reviews_des.head(10)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,sentiment,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-11-05,,1250,No ratings yet,1,simple yet great replayability in opinion zomb...,0.211111,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-07-15,,22200,No ratings yet,1,its unique worth playthrough,0.337500,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-04-21,,43110,No ratings yet,1,great atmosphere the gunplay bit chunky times ...,0.075000,2
3,js41637,http://steamcommunity.com/id/js41637,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,1,i know think see title barbie dreamhouse party...,0.114583,2
4,js41637,http://steamcommunity.com/id/js41637,,2013-09-08,,227300,0 of 1 people (0%) found this review helpful,1,for simple its actually simple be truck drivin...,-0.037917,0
5,js41637,http://steamcommunity.com/id/js41637,,2013-11-29,,239030,1 of 4 people (25%) found this review helpful,1,very fun little game play bored time passer ve...,-0.099500,0
6,evcentric,http://steamcommunity.com/id/evcentric,,NaT,,248820,No ratings yet,1,a suitably punishing roguelike platformer winn...,0.231548,2
7,evcentric,http://steamcommunity.com/id/evcentric,,2015-12-04,"Last edited December 5, 2015.",370360,No ratings yet,1,run fun what hell kind fun that,0.400000,2
8,evcentric,http://steamcommunity.com/id/evcentric,,2014-11-03,,237930,No ratings yet,1,elegant integration gameplay story world devel...,0.500000,2
9,evcentric,http://steamcommunity.com/id/evcentric,,2014-10-15,,263360,No ratings yet,1,random drops random quests stat points animati...,-0.333333,0


Veo cuanto nulos hay en la columna posted

In [74]:
df_reviews_des['posted'].isnull().sum()

9932

Elimino filas con nulos en la columna 'posted'

In [81]:
df_reviews_des.dropna(subset=['posted'], inplace=True)

Teniendo en cuenta las consultas que se deben resolver decido eliminar algunas columnas

In [77]:
df_reviews_des.drop(['user_url', 
                     'funny', 
                     'last_edited', 
                     'helpful', 
                     'review', 
                     'sentiment'], axis=1, inplace=True)

In [82]:
df_reviews_des.head(10)

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,2011-11-05,1250,1,2
1,76561197970982479,2011-07-15,22200,1,2
2,76561197970982479,2011-04-21,43110,1,2
3,js41637,2014-06-24,251610,1,2
4,js41637,2013-09-08,227300,1,0
5,js41637,2013-11-29,239030,1,0
7,evcentric,2015-12-04,370360,1,2
8,evcentric,2014-11-03,237930,1,2
9,evcentric,2014-10-15,263360,1,0
10,evcentric,2014-10-15,107200,1,2


In [83]:
df_reviews_des.shape

(48498, 5)

Guardo el archivo como reviews.parquet

In [ ]:
savefile_path = 'datasets/reviews.parquet'
df_reviews_des.to_parquet(savefile_path, engine='pyarrow')

# Archivos para los outputs

In [84]:
games = pd.read_parquet('datasets/games.parquet')
items = pd.read_parquet('datasets/items.parquet')
reviews = pd.read_parquet('datasets/reviews.parquet')

In [85]:
games.head(1)

,title,release_date,price,item_id,Accounting,Action,Adventure,Animation&amp;Modeling,AudioProduction,Casual,...,PhotoEditing,RPG,Racing,Simulation,SoftwareTraining,Sports,Strategy,Utilities,VideoProduction,WebPublishing
0,Lost Summoner Kitty,2018-01-04,4.99,761140,0,1,0,0,0,1,...,0,0,0,1,0,0,1,0,0,0


In [86]:
items.head(1)

,user_id,item_id,playtime_forever
0,76561197970982479,10,6


In [87]:
reviews.head(1)

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,2011-11-05,1250,1,2


Función 1

In [ ]:
aux = pd.merge(items, games, on='item_id')

In [ ]:
aux = aux.drop(['title', 'price', 'user_id', 'item_id'], axis=1)

In [ ]:
aux.head()

In [ ]:
savefile_path = 'func_1.parquet'
aux.to_parquet(savefile_path, engine='pyarrow')

Función 2

In [ ]:
aux2 = aux.drop(['title', 'price', 'item_id'], axis=1)

In [ ]:
savefile_path = 'func_2.parquet'
aux2.to_parquet(savefile_path, engine='pyarrow')

funcion 3

In [96]:
aux3 = games[['title', 'release_date', 'item_id']]
aux3_1 = reviews[['item_id', 'recommend', 'sentiment_analysis']]

In [98]:
aux3['release_date'] = aux3['release_date'].dt.year

C:\Users\dines\AppData\Local\Temp\ipykernel_15328\4031569698.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux3['release_date'] = aux3['release_date'].dt.year


In [99]:
func_3 = pd.merge(aux3_1, aux3, on='item_id')

In [100]:
func_3.drop(['item_id'], axis=1, inplace=True)

In [102]:
func_3 = func_3[func_3['recommend'] == 1]
func_3 = func_3[func_3['sentiment_analysis'].isin([1, 2])]

In [104]:
savefile_path = 'func_3.parquet'
func_3.to_parquet(savefile_path, engine='pyarrow')

Función 4

In [123]:
aux4 = games[['title', 'release_date', 'item_id']]
aux4_1 = reviews[['item_id', 'recommend', 'sentiment_analysis']]

In [124]:
aux4['release_date'] = aux4['release_date'].dt.year

C:\Users\dines\AppData\Local\Temp\ipykernel_15328\1444849050.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux4['release_date'] = aux4['release_date'].dt.year


In [125]:
func_4 = pd.merge(aux4_1, aux4, on='item_id')

In [126]:
func_4.drop(['item_id'], axis=1, inplace=True)

In [127]:
func_4 = func_4[func_4['recommend'] == 0]
func_4 = func_4[func_4['sentiment_analysis'] == 0]

In [128]:
savefile_path = 'func_4.parquet'
func_4.to_parquet(savefile_path, engine='pyarrow')

Función 5

In [121]:
aux5 = games[['title', 'release_date', 'item_id']]
aux5_1 = reviews[['item_id', 'recommend', 'sentiment_analysis']]

In [122]:
aux5['release_date'] = aux5['release_date'].dt.year


C:\Users\dines\AppData\Local\Temp\ipykernel_15328\399874914.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux5['release_date'] = aux5['release_date'].dt.year


In [129]:
func_5 = pd.merge(aux5_1, aux5, on='item_id')

In [130]:
func_5.drop(['item_id'], axis=1, inplace=True)

In [131]:
savefile_path = 'func_5.parquet'
func_5.to_parquet(savefile_path, engine='pyarrow')

Recomendacion